<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/512_aydao_gpu_shawwn_Stylegan2_Custom_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://sdk.cloud.google.com | bash

Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
#!gcloud init

In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-aydao-gpu"):
    %cd "/content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-aydao-gpu
    %cd colab-sg2-aydao-gpu
    !git clone https://github.com/shawwn/stylegan2.git
#    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2
    !mkdir downloads
    !mkdir datasets

/content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2


In [ ]:
%cd "/content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
#!git fetch origin
#!git checkout origin/main -- train.py

/content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2


## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [4]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=MGrUCmriIxBJxIQm4nI79kwjYwC03Y&prompt=consent&access_type=offline&code_challenge=dNLpyZcFiiFXaZxpKoDXrY-PnxSZSkTgap6rs7JpZWw&code_challenge_method=S256

Enter verification code: 4/1AX4XfWigMhSmjh2mZ3ZdLPfoKtLTTwidVvujt3nSOjFOZiZC-VrK6-stUBM

You are now logged in as [train.colab.tpu@neuroidss.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


To take a quick anonymous survey, run:
  $ gcloud survey



In [5]:
#project_id = 'encoded-phalanx-326615'
project_id = 'local-abbey-335821'
!gcloud config set project {project_id}

Updated property [core/project].


In [6]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/sq-512.zip"

!unzip {zip_path} -d /content/

Archive:  /content/drive/MyDrive/sq-512.zip
   creating: /content/sq-512/
  inflating: /content/sq-512/3870008_1.jpg  
  inflating: /content/sq-512/2235011.jpg  
  inflating: /content/sq-512/3912003.jpg  
  inflating: /content/sq-512/1064013.jpg  
  inflating: /content/sq-512/3093010.jpg  
  inflating: /content/sq-512/307004.jpg  
  inflating: /content/sq-512/633007.jpg  
  inflating: /content/sq-512/2503003.jpg  
  inflating: /content/sq-512/1700004.jpg  
  inflating: /content/sq-512/2699009.jpg  
  inflating: /content/sq-512/2122011.jpg  
  inflating: /content/sq-512/1740001.jpg  
  inflating: /content/sq-512/3877012.jpg  
  inflating: /content/sq-512/1508007.jpg  
  inflating: /content/sq-512/288011.jpg  
  inflating: /content/sq-512/1044008_0.jpg  
  inflating: /content/sq-512/2944008.jpg  
  inflating: /content/sq-512/187006.jpg  
  inflating: /content/sq-512/1444013.jpg  
  inflating: /content/sq-512/1419000.jpg  
  inflating: /content/sq-512/343005.jpg  
  inflating: /content/sq

In [7]:
#update this to the path to your image folder
dataset_path = "/content/sq-512"
#give your dataset a name
dataset_name = "sq-512"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}
#!python dataset_tool.py create_from_images gs://train_with_tpu/datasets/{dataset_name} {dataset_path}

Loading images from "/content/sq-512"
Creating dataset "./datasets/sq-512"
dataset_tool.py:86: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 670 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
  import os
  import tensorflow.compat.v1 as tf
  tf.disable_eager_execution()
  import pprint
  assert 'COLAB_TPU_ADDR' in os.environ, 'Did you forget to switch to TPU?'
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(tpu_address) as sess:
    devices = sess.list_devices()
  pprint.pprint(devices)
  device_is_tpu = [True if 'TPU' in str(x) else False for x in devices]
  assert True in device_is_tpu, 'Did you forget to switch to TPU?'


In [ ]:
print(tpu_address)

grpc://10.5.216.90:8470


In [ ]:
project_id = 'encoded-phalanx-326615'
!gcloud config set project {project_id}

Updated property [core/project].


In [8]:
!gsutil cp -r ./datasets/* gs://train_with_tpu2/datasets/


Copying file://./datasets/sq-512/sq-512-r02.tfrecords [Content-Type=application/octet-stream]...
Copying file://./datasets/sq-512/sq-512-r04.tfrecords [Content-Type=application/octet-stream]...
Copying file://./datasets/sq-512/sq-512-r05.tfrecords [Content-Type=application/octet-stream]...
Copying file://./datasets/sq-512/sq-512-r03.tfrecords [Content-Type=application/octet-stream]...
- [4 files][  2.7 MiB/  2.7 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./datasets/sq-512/sq-512-r06.tfrecords [Content-Type=application/octet-stream]...
Copying file://./datasets/sq-512/sq-512-r07.tfrecords [Content-Type=application/octet-stream]...
Copying file://./datasets/sq-512/sq-512-r09.tfrecords [Content-Type=application/octet

In [ ]:
!gsutil cp -r /content/drive/MyDrive/colab-sg2/stylegan2/datasets/* gs://train_with_tpu/datasets/


In [ ]:
GCP_PROJECT_ID = 'encoded-phalanx-326615'
PROJECT_NUMBER = '0'


In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com


Operation "operations/acf.p2-174339478235-bf3cf8dc-daf5-4715-bb29-97389dd24333" finished successfully.


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
#!export TPU_NAME={tpu_address}
#!export TPU_NAME=""
#!export RESUME_PKL="/content/drive/MyDrive/sg2-ext_aydao-anime-danbooru2019s-512-5268480.pkl"
#!export RESUME_PKL=./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl
#!export RESUME_KIMG=100.4
#!export RESUME_TIME=43
#!export RESUME_TIME=`math 10*3600 + 13*60 + 49` # 10h 13m 49s
#RESUME_TIME=0*3600 + 44*60 + 31
#!export RESUME_TIME={RESUME_TIME} # 10h 13m 49s

#    resume_pkl="./results/00019-stylegan2-sq-256-1gpu-config-a/network-snapshot-000100.pkl"
#    resume_kimg=100.4
#    resume_time=0*3600 + 44*60 + 31
#    resume_pkl="/content/drive/MyDrive/stylegan2-cat-config-a.pkl"
#    resume_kimg=0
#    resume_time=0

#config="config-f" # StyleGAN 2
config="config-a" # StyleGAN 1

#data_dir="gs://train_with_tpu/datasets"
data_dir="./datasets"
dataset="sq-512"
mirror="False"
metrics="none"
#!export NO_GCE_CHECK=true
#total_kimg=5268480+1000 #25000
total_kimg=25000
#resume="./results/00042-stylegan2-sq-512-1gpu-config-a"
#resume="gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl"
#resume="/content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl"

#!python run_training.py --resume "{resume}" --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"
!python run_training.py --total-kimg "{total_kimg}" --data-dir "{data_dir}" --config="{config}" --dataset="{dataset}" --mirror-augment="{mirror}" --metrics="{metrics}"


In [ ]:
!python convert_ckpt_to_pkl.py --tpu_address={tpu_address} --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-512 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl

In [ ]:
!gsutil cp -r gs://train_with_tpu/networks_aydao/model.ckpt-9728.pkl /content/drive/MyDrive/networks_aydao/model.ckpt-latest.pkl

In [ ]:
!python convert_ckpt_to_pkl.py --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-512 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl

python3: can't open file 'convert_ckpt_to_pkl.py': [Errno 2] No such file or directory


In [ ]:
!gsutil cp -r gs://train_with_tpu/networks_aydao/model.ckpt-9728.pkl /content/drive/MyDrive/networks_aydao/model.ckpt-latest.pkl

In [ ]:
!python convert_ckpt_to_pkl.py --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-512 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl

python3: can't open file 'convert_ckpt_to_pkl.py': [Errno 2] No such file or directory


### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.